In [ ]:
import config
import json
import pickle
import time
import os
## set seeds
from utils_picking import round_percent
import random
random.seed(1234)
import numpy as np; np.random.seed(1234)


In [ ]:
# MAKE CHANGES HERE
MODEL = 'SNMN'   # alternatively can put "SAAA", "CNN_LSTM" 

In [ ]:
TEST_SPLIT = 'val2014'
orig_root_dir_qa = config.cv_qa_test_dir_orig
edit_root_dir_qa = config.cv_qa_test_dir_edit

#### questions keys: 'image_id', 'question', 'question_id'
res_file_q = 'v2_OpenEnded_mscoco_' + TEST_SPLIT + '_questions.json'
standard_questions_val_json = os.path.join(orig_root_dir_qa, res_file_q)
standard_questions_edit_val_json = os.path.join(edit_root_dir_qa, res_file_q)

## ann keys: 'image_id', 'question_id', 'answers' , 'multiple_choice_answer'(the most frequent answer), 'question_type', 'answer_type'
res_file_a = 'v2_mscoco_' + TEST_SPLIT + '_annotations.json'
standard_annotations_val_json = os.path.join(orig_root_dir_qa, res_file_a)
standard_annotations_edit_val_json = os.path.join(edit_root_dir_qa, res_file_a)

In [ ]:
if MODEL == 'SNMN':
    results_edit_val= config.results_edit_val_snmn_cv_vqa
    results_val= config.results_val_snmn
    ## standard_vocab file_model_specific
    standard_vocab_ans_file = config.standard_vocab_ans_file_snmn
    with open(standard_vocab_ans_file) as f:
        ans_vocab_list = f.read().splitlines()
    
elif MODEL == 'SAAA':
    results_edit_val = config.results_edit_val_saaa_cv_vqa
    results_val = config.results_val_saaa
    ## standard_vocab file_model_specific
    vocab = config.standard_vocab_ans_file_saaa
    with open(vocab, 'r') as f:
        ans_vocab = json.load(f)["answer"]
        ans_vocab_list = [k for k, v in ans_vocab.items()]
        #{v: k for k, v in ans_vocab.items()}   ### is a dictionary here but will work: keys- index- 0,1,2...

elif MODEL == 'CNN_LSTM':
    results_edit_val = config.results_edit_val_cnn_lstm_cv_vqa
    results_val = config.results_val_cnn_lstm
    ## standard_vocab file_model_specific
    vocab = config.standard_vocab_ans_file_cnn_lstm
    with open(vocab, 'r') as f:
        ans_vocab = json.load(f)["answer"]
        ans_vocab_list = [k for k, v in ans_vocab.items()]
        # {v: k for k, v in ans_vocab.items()}   ### is a dictionary here but will work: keys- index- 0,1,2...
    


In [ ]:
def my_read_old(results_old_pkl, standard_q_json, standard_a_json=None):
    st = time.time()
    with open(results_old_pkl, 'rb') as file:
        res_val = pickle.load(file)
    if str(standard_q_json)[-4:] == 'json':
        with open(standard_q_json) as file:
            details_ques = json.load(file)['questions']
    else:
        with open(standard_q_json, 'rb') as file:
            details_ques = pickle.load(file)['questions']

    ## from results file- model specific
    pred_ans = [details['ans_id'] for details in res_val]
    softmax_vector = [details['ss_vc'] for details in res_val]
    img_id_res = [details['img_id'] for details in res_val]
    qid_res = [details['ques_id'] for details in res_val]
    # gt_ans_used = [details['gt_ans_id_used'] for details in res_val] ## in case of snmn
    # q_ids = [details['ques_id'] for details in res_val]     ### in cases of snmn
    # assert (q_ids == [details_q['question_id'] for details_q in details_ques])

    q_ids = [details_q['question_id'] for details_q in details_ques]
    img_ids = [details_q['image_id'] for details_q in details_ques]
    img_ids = [str(iid).zfill(12) for iid in img_ids]
    ques_str = [details_q['question'] for details_q in details_ques]

    ### make sure order of qid, img_id consistent between results and standard annotations files
    ## if not, sort them to what is there in the standard files
    #ipdb.set_trace()
    if qid_res != q_ids and img_id_res != img_ids:
        std_q_img_id = {}

        ### SAA and CNNN_LSTM in case for train set: we find a discrpancy so qids are nto a strict subset of qid_res
        if len((set(q_ids) & set(qid_res))) != len(set(q_ids)):
            print()
            print(' passing wrong file- please check the std_ques and results files: DISCREPANCY ALERT')
            print()
            target_q_ids = list((set(q_ids) & set(qid_res)))
        else:    ## ideally q_ids is a strict subset of res_qids...
            target_q_ids = q_ids

        for qid_idx, qid in enumerate(target_q_ids):
            key = str(qid) +  str(img_ids[qid_idx])
            std_q_img_id[key] = qid_idx

        res_q_img_id_pans_ss_vc = {}
        for qid_idx, qid in enumerate(qid_res):
            # if 'torch' in img_id_res[qid_idx].type():
            #     img_id_res[qid_idx] = img_id_res[qid_idx].item()
            # if len(str(img_id_res[qid_idx])) < 20:  ## SAAA fix; for edit_set- img_id is string and not torch object
            #     img_id_res[qid_idx] = img_id_res[qid_idx].item()
            key_new = str(qid) + str(img_id_res[qid_idx])
            res_q_img_id_pans_ss_vc[key_new] = pred_ans[qid_idx], softmax_vector[qid_idx], qid, img_id_res[qid_idx]

        pred_ans_corr = [res_q_img_id_pans_ss_vc[key][0] for key in std_q_img_id.keys()]
        softmax_vector_corr = [res_q_img_id_pans_ss_vc[key][1] for key in std_q_img_id.keys()]
        qid_corr = [res_q_img_id_pans_ss_vc[key][2] for key in std_q_img_id.keys()]
        img_id_corr = [res_q_img_id_pans_ss_vc[key][3] for key in std_q_img_id.keys()]
        assert img_id_corr == img_ids
        assert qid_corr == q_ids

        pred_ans = pred_ans_corr
        softmax_vector = softmax_vector_corr


    if standard_a_json is not None:
        with open(standard_a_json) as file:
            details_ann = json.load(file)['annotations']
        all_answers = [[ans['answer'] for ans in details_ann[i]['answers']] for i in range(len(img_ids))]
        # most_freq_ans = [details_ann[i]['multiple_choice_answer'] for i in range(len(img_ids))]
        ques_type_data = [details_ann[i]['question_type'] for i in range(len(img_ids))]
        ans_type_data = [details_ann[i]['answer_type'] for i in range(len(img_ids))]
        print(time.time() - st)
        return q_ids, pred_ans, softmax_vector, img_ids, ques_str, all_answers, ques_type_data, ans_type_data     # ,gt_ans_used
    else:
        print(time.time() - st)
        return q_ids, pred_ans, softmax_vector, img_ids, ques_str, [], [], []  # ,gt_ans_used

In [ ]:
qid_val, pred_ans_val , ss_vc_val, img_ids_val, ques_val, all_ans_val, ques_type_val, ans_type_val = my_read_old(results_val, standard_questions_val_json, standard_annotations_val_json)

qid_edit_val, pred_ans_edit_val , ss_vc_edit_val, img_ids_edit_val, ques_edit_val, all_ans_edit_val, ques_type_edit_val, ans_type_edit_val = my_read_old(results_edit_val, standard_questions_edit_val_json, standard_annotations_edit_val_json)

In [ ]:
### since we are taking just val_90_10: for testing purpose here: so there are some del_1 ques_ids who dont have
## corresponding original IQAs because we just took the first 90 split of val.

## BETTER TO RESTRICT THE QID_EDIT_VAL!!!

### HARD-CODED!
# print('REMINDER: only for val_90_10, just taking first 3743 indices of del_1, this is hard-coded')
# assert [qid_edit_val[i] not in qid_ss_predans_val.keys() for i in range(3743,len(qid_edit_val),1)]
# qid_edit_val = qid_edit_val[0:3743]

# chunk_these_lists = [qid_edit_val, pred_ans_edit_val , ss_vc_edit_val, img_ids_edit_val, ques_edit_val, all_ans_edit_val, ques_type_edit_val, ans_type_edit_val]
# chunked_lists = [i[0:3743] for i in chunk_these_lists]
# qid_edit_val, pred_ans_edit_val , ss_vc_edit_val, img_ids_edit_val, ques_edit_val, all_ans_edit_val, ques_type_edit_val, ans_type_edit_val = chunked_lists


masking_indices_where_qid_edit_but_no_orig_qid = [idx for idx, i in enumerate(qid_edit_val) if i not in qid_val]
stop_idx = masking_indices_where_qid_edit_but_no_orig_qid[0]
qid_edit_val = qid_edit_val[0:stop_idx]
qid_edit_val, pred_ans_edit_val , ss_vc_edit_val, img_ids_edit_val, ques_edit_val, all_ans_edit_val, ques_type_edit_val, ans_type_edit_val = [i[0:stop_idx] for i in [qid_edit_val, pred_ans_edit_val , ss_vc_edit_val, img_ids_edit_val, ques_edit_val, all_ans_edit_val, ques_type_edit_val, ans_type_edit_val]]


len_val = len(img_ids_val)
len_edit_val = len(img_ids_edit_val)
all_indices_val = np.arange(len_val)
all_indices_edit_val = np.arange(len_edit_val)

print('size of original set:', len_val)
print('size of edited set:', len_edit_val)
print('no of unique images in val_set: ',len(set(img_ids_val)))
print('no of unique images in edit_val_set: ',len(set(img_ids_edit_val)))
print('#unique questions in original set: ',len(list(set(qid_val))) )
print('#unique questions in edited set: ', len(list(set(qid_edit_val))))

if len(list(set(qid_val))) != len(list(set(qid_edit_val))) :
    print('not every question in orig_val made it to edit_val=> for ', len(list(set(qid_val)))- len(list(set(qid_edit_val)))  ,' questions- no legit edited IQA possible')


In [ ]:
qid_gt_ans_label = {}
for idx,a in enumerate(qid_val):
    qid_gt_ans_label[a] = all_ans_val[idx]
    
qid_predans_val = {}
for idx, a in enumerate(qid_val):
    qid_predans_val.setdefault(a, []).append(pred_ans_val[idx])
    
qid_predans_idx_val = {}
for idx, a in enumerate(qid_val):
    qid_predans_idx_val.setdefault(a, []).append(idx)
    
#idx in case here refers to len(val and edit_val- order hai - so relax)    
qid_predans_edit_val = {}
for idx, a in enumerate(qid_edit_val):
    qid_predans_edit_val.setdefault(a, []).append(pred_ans_edit_val[idx])
    #qid_predans_edit_val[a] = (pred_ans_edit_val[idx])   
    
qid_predans_idx_edit_val = {}
for idx, a in enumerate(qid_edit_val):
    qid_predans_idx_edit_val.setdefault(a, []).append(idx)    
    
# qid_predans_imgid_edit_val = {}
# for idx, a in enumerate(qid_edit_val):
#     qid_predans_imgid_edit_val.setdefault(a, []).append(img_ids_edit_val[idx])   

## creating dictionary for val set - to facilitate extensions based on q_id index
qid_ss_predans_val = {}
for idx, a in enumerate(qid_val):
    qid_ss_predans_val[a] = (ss_vc_val[idx], pred_ans_val[idx], qid_val[idx],idx, all_ans_val[idx] )

extended_ss_vc_val = [qid_ss_predans_val[q_id][0] for q_id in qid_edit_val ]
extended_pred_ans_val = [qid_ss_predans_val[q_id][1] for q_id in qid_edit_val ] 
extended_gt_ans_val = [qid_gt_ans_label[q_id][0] for q_id in qid_edit_val ] 
extended_qid_val = [qid_ss_predans_val[q_id][2] for q_id in qid_edit_val]

collapsed_pred_ans_val= [qid_ss_predans_val[q_id][1] for q_id in set(qid_edit_val) ] 

collapsed_gt_ans_val = [qid_ss_predans_val[q_id][4][0] for q_id in set(qid_edit_val)] 
collapsed_indices = [qid_ss_predans_val[q_id][3] for q_id in set(qid_edit_val)] 

gt_ans_edit_val = [i[0] for i in all_ans_edit_val]
gt_ans_val = [i[0] for i in all_ans_val]
### all_10_ans same
## extended_gt_ans_edit_val
## extended_gt_ans_val


assert [int(gt_ans_edit_val[i])+1== int(extended_gt_ans_val[i]) for i in range(len(gt_ans_edit_val))]

assert len(collapsed_indices) == len(set(qid_edit_val))


##### pred: predictions: need to  be mapped to ans_vocab
####pred_ans/gt_ans: has to be string!!!!!!

In [ ]:
###                                         STATISTICS

accuracy_ind_before = [i for i in range(len_val) if ans_vocab_list[pred_ans_val[i]] == gt_ans_val[i]]
accuracy_ind_before_extended = [i for i in range(len_edit_val) if ans_vocab_list[extended_pred_ans_val[i]]== extended_gt_ans_val[i]]
accuracy_ind_after = [i for i in range(len_edit_val) if ans_vocab_list[pred_ans_edit_val[i]] == gt_ans_edit_val[i]]
accuracy_ind_before_collapsed = [i for i in range(len(set(qid_edit_val))) if ans_vocab_list[collapsed_pred_ans_val[i]] == collapsed_gt_ans_val[i]]
print('accuracy before editing, one answer match is good', 
      round_percent(len(accuracy_ind_before) / len_val), len(accuracy_ind_before),'/', len_val)
print('accuracy before editing_extended', 
      round_percent(len(accuracy_ind_before_extended) / len_edit_val), len(accuracy_ind_before_extended), '/', len_edit_val)
print('accuracy after editing, one answer match is good', 
      round_percent(len(accuracy_ind_after) / len_edit_val), len(accuracy_ind_after),'/', len_edit_val)

print('accuracy before editing_collapsed', 
      round_percent(len(accuracy_ind_before_collapsed) / len(set(qid_edit_val))), len(accuracy_ind_before_collapsed),'/', len(set(qid_edit_val)))



In [ ]:
chuck_many_list = ['many', 'black']
chuck_many = [i for i, val in enumerate(pred_ans_edit_val) if ans_vocab_list[val] in chuck_many_list]

print('many there in predicted answers, so we would remove them; #many: ', len(chuck_many))

if len(chuck_many) != 0:
    ### for entire set counting how mnay labels flipped
    labels_flipped_count = np.sum([ans_vocab_list[extended_pred_ans_val[i]] != str(int(ans_vocab_list[val]) + 1)
                                   if ans_vocab_list[val] not in chuck_many_list else 0 for i, val in
                                   enumerate(pred_ans_edit_val)])
    # labels_flipped_count = np.sum([ans_vocab_list[extended_pred_ans_val[i]] != str(int(ans_vocab_list[val])+1) for i, val in enumerate(pred_ans_edit_val)])
    labels_remained_same_count = np.sum(
        [ans_vocab_list[extended_pred_ans_val[i]] == str(int(ans_vocab_list[val]) + 1)
         if ans_vocab_list[val] not in chuck_many_list else 0 for i, val in enumerate(pred_ans_edit_val)])
    assert (labels_flipped_count + labels_remained_same_count + len(chuck_many) == len(pred_ans_edit_val))

    indices_not_many = [i for i, val in enumerate(pred_ans_edit_val) if ans_vocab_list[val] not in chuck_many_list]

    lab_fl_ind = [i for i in indices_not_many if ans_vocab_list[extended_pred_ans_val[i]]
                  != str(int(ans_vocab_list[pred_ans_edit_val[i]]) + 1)]
    lab_fl_pos = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] != extended_gt_ans_val[i]
                  and ans_vocab_list[pred_ans_edit_val[i]] == gt_ans_edit_val[i]]
    lab_fl_neg = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] == extended_gt_ans_val[i]
                  and ans_vocab_list[pred_ans_edit_val[i]] != gt_ans_edit_val[i]]
    lab_fl_right = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] == extended_gt_ans_val[i]
                    and ans_vocab_list[pred_ans_edit_val[i]] == gt_ans_edit_val[i]]
    lab_fl_wrong = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] != extended_gt_ans_val[i]
                    and ans_vocab_list[pred_ans_edit_val[i]] != gt_ans_edit_val[i]]
    assert (len(lab_fl_pos) + len(lab_fl_neg) + len(lab_fl_right) + len(lab_fl_wrong) == len(lab_fl_ind))

    lab_sm_ind = [i for i in indices_not_many if ans_vocab_list[extended_pred_ans_val[i]]
                  == str(int(ans_vocab_list[pred_ans_edit_val[i]]) + 1)]
    lab_sm_right = [i for i in lab_sm_ind if ans_vocab_list[extended_pred_ans_val[i]] == extended_gt_ans_val[i]
                    and ans_vocab_list[pred_ans_edit_val[i]] == gt_ans_edit_val[i]]
    lab_sm_wrong = [i for i in lab_sm_ind if ans_vocab_list[extended_pred_ans_val[i]] != extended_gt_ans_val[i]
                    and ans_vocab_list[pred_ans_edit_val[i]] != gt_ans_edit_val[i]]
    assert (len(lab_sm_right) + len(lab_sm_wrong) == len(lab_sm_ind))


else:
    ### for entire set counting how mnay labels flipped
    labels_flipped_count = np.sum(
        [ans_vocab_list[extended_pred_ans_val[i]] != str(int(ans_vocab_list[val]) + 1) for i, val in
         enumerate(pred_ans_edit_val)])
    # labels_flipped_count = np.sum([ans_vocab_list[extended_pred_ans_val[i]] != str(int(ans_vocab_list[val])+1) for i, val in enumerate(pred_ans_edit_val)])
    labels_remained_same_count = np.sum(
        [ans_vocab_list[extended_pred_ans_val[i]] == str(int(ans_vocab_list[val]) + 1) for i, val in
         enumerate(pred_ans_edit_val)])
    assert (labels_flipped_count + labels_remained_same_count == len(pred_ans_edit_val))

    lab_fl_ind = [i for i in range(len_edit_val) if ans_vocab_list[extended_pred_ans_val[i]]
                  != str(int(ans_vocab_list[pred_ans_edit_val[i]]) + 1)]

    lab_fl_pos = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] != extended_gt_ans_val[i]
                  and ans_vocab_list[pred_ans_edit_val[i]] == gt_ans_edit_val[i]]
    lab_fl_neg = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] == extended_gt_ans_val[i]
                  and ans_vocab_list[pred_ans_edit_val[i]] != gt_ans_edit_val[i]]
    lab_fl_right = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] == extended_gt_ans_val[i]
                    and ans_vocab_list[pred_ans_edit_val[i]] == gt_ans_edit_val[i]]
    lab_fl_wrong = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] != extended_gt_ans_val[i]
                    and ans_vocab_list[pred_ans_edit_val[i]] != gt_ans_edit_val[i]]
    assert (len(lab_fl_pos) + len(lab_fl_neg) + len(lab_fl_right) + len(lab_fl_wrong) == len(lab_fl_ind))

    lab_sm_ind = [i for i in range(len_edit_val) if ans_vocab_list[extended_pred_ans_val[i]]
                  == str(int(ans_vocab_list[pred_ans_edit_val[i]]) + 1)]
    lab_sm_right = [i for i in lab_sm_ind if ans_vocab_list[extended_pred_ans_val[i]] == extended_gt_ans_val[i]
                    and ans_vocab_list[pred_ans_edit_val[i]] == gt_ans_edit_val[i]]
    lab_sm_wrong = [i for i in lab_sm_ind if ans_vocab_list[extended_pred_ans_val[i]] != extended_gt_ans_val[i]
                    and ans_vocab_list[pred_ans_edit_val[i]] != gt_ans_edit_val[i]]
    assert (len(lab_sm_right) + len(lab_sm_wrong) == len(lab_sm_ind))

In [ ]:
## all final answers in string. Dont forget to match the ans+predicted to the vocab used!
### for entire set counting how mnay labels flipped

print('#labels not n/n-1 conssitent for entire set=', labels_flipped_count,'      ' ,
      round_percent(labels_flipped_count / len_edit_val), '%')

print('#labels n/n-1 conssitent =', labels_remained_same_count,'       ', 
      round_percent(labels_remained_same_count / len_edit_val), '%')


print('neg->pos', len(lab_fl_pos), '         ', 
      round_percent(len(lab_fl_pos) / len_edit_val), '%' )
print('pos->neg', len(lab_fl_neg), '         ', 
      round_percent(len(lab_fl_neg) / len_edit_val), '%')
print('pos->pos', len(lab_fl_right), '         ', round_percent(len(lab_fl_right) / len_edit_val), '%')
print('neg->neg', len(lab_fl_wrong), '         ', round_percent(len(lab_fl_wrong) / len_edit_val), '%')



# labels that remained exactly same- so two cases possible- either right/wrong

print('#labels consistent both right', len(lab_sm_right), '         ', round_percent(len(lab_sm_right)/len_edit_val), '%')
print('#labels consistent both wrong', len(lab_sm_wrong), '         ', round_percent(len(lab_sm_wrong) / len_edit_val), '%')